In [1]:
from random import Random
import numpy as np
import pandas as pd
from tva.situation import Situation
from tva.strategies import Strategies
import copy
from tva.schemes import Schemes, VotingScheme
from tva.voter import Voter

In [2]:
# Create the voting situation
situation = Situation(num_voters=4, num_candidates=4)
display(situation)

# Apply the voting scheme to the situation
schemes = Schemes()
voting_scheme = VotingScheme.BORDA
schemes.print_results(situation, verbose=True)



Voter 0: ['B', 'A', 'D', 'C']
Voter 1: ['B', 'D', 'C', 'A']
Voter 2: ['D', 'A', 'C', 'B']
Voter 3: ['B', 'C', 'A', 'D']

Anti plurality: B {'B': 3, 'A': 3, 'D': 3, 'C': 3} 
Two voting: B {'B': 3, 'A': 2, 'D': 2, 'C': 1} 
Borda: B {'B': 9, 'A': 5, 'D': 6, 'C': 4}


In [28]:
situation = Situation(num_voters=4, num_candidates=4)
situation.voters[0].preferences = ["A","B","D","C"]
situation.voters[1].preferences = ["B","C","A","D"]
situation.voters[2].preferences = ["A","D","B","C"]
situation.voters[3].preferences = ["B","C","D","A"]
situation

Voter 0: ['A', 'B', 'D', 'C']
Voter 1: ['B', 'C', 'A', 'D']
Voter 2: ['A', 'D', 'B', 'C']
Voter 3: ['B', 'C', 'D', 'A']

In [32]:

display(situation)
winnner, scores = schemes.borda_voting(situation.voters, True)
print("Borda:", winnner, scores)
situation2 = copy.deepcopy(situation)
situation2.voters[0].preferences = ["A"]
winnner, scores = schemes.borda_voting(situation2.voters, True)
display(situation2)
print("Borda:", winnner, scores)

Voter 0: ['A', 'B', 'D', 'C']
Voter 1: ['B', 'C', 'A', 'D']
Voter 2: ['A', 'D', 'B', 'C']
Voter 3: ['B', 'C', 'D', 'A']

Borda: B {'A': 7, 'B': 9, 'D': 4, 'C': 4}


Voter 0: ['A']
Voter 1: ['B', 'C', 'A', 'D']
Voter 2: ['A', 'D', 'B', 'C']
Voter 3: ['B', 'C', 'D', 'A']

Borda: B {'A': 4, 'B': 7, 'C': 4, 'D': 3}


Create 1000 situations and check for what fraction of them a strategy is good

We do experiments for each voting scheme separately.<br>
TODO: We need to save, what type of strategy has won this time?

In [33]:
strategies = Strategies()
voting_scheme = VotingScheme.BORDA
nr_repetitions = 1000
num_voters = 4

strategy_counter = 0
for i in range(nr_repetitions):
    situation = Situation(num_voters=num_voters, num_candidates=4)
    # Check if at least one voter has a good strategy
    for voter_index in range(num_voters):
        if strategies.is_any_strategy_good(situation, voter_index, voting_scheme):
            strategy_counter += 1
            break
        
print(strategy_counter/nr_repetitions)

0.877


Plurality cannot be affected by strategic voting because that would require a voter to vote for a candidate they prefer less than their favorite candidate.